In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import polars as pl

sns.set_palette("deep")
import warnings

warnings.filterwarnings("ignore")

In [3]:
ROOT_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))
ROOT_PATH

'/home/mlia/proyectos/data-generation'

In [4]:

DATA_PATH = os.path.join(ROOT_PATH, "data", "processed")
EMT_DATA_PATH = os.path.join(DATA_PATH, "emt")
EMT_DATA_PATH

'/home/mlia/proyectos/data-generation/data/processed/emt'

In [5]:
df = pl.read_csv(os.path.join(EMT_DATA_PATH, "2024", "03", "emt_202403.csv"))

In [6]:
df.head()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive
str,str,i64,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,bool,str,str,i64
"""2024-03-02""","""2024-03-02 07:…",579,"""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",1409
"""2024-03-02""","""2024-03-02 07:…",581,"""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",476
"""2024-03-02""","""2024-03-02 07:…",4855,"""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,false,null,null,2481
"""2024-03-02""","""2024-03-02 07:…",4857,"""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,false,null,null,967
"""2024-03-02""","""2024-03-02 07:…",5631,"""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,false,null,null,899


In [ ]:
## VVNC

### Distribuciones

In [ ]:
var_name = "DistanceBus"

In [ ]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df[var_name].describe())


Estadísticas descriptivas:


shape: (9, 2)
┌────────────┬─────────────┐
│ statistic  ┆ value       │
│ ---        ┆ ---         │
│ str        ┆ f64         │
╞════════════╪═════════════╡
│ count      ┆ 2.7659028e7 │
│ null_count ┆ 0.0         │
│ mean       ┆ 3382.528037 │
│ std        ┆ 3099.132083 │
│ min        ┆ -20433.0    │
│ 25%        ┆ 1133.0      │
│ 50%        ┆ 2580.0      │
│ 75%        ┆ 4704.0      │
│ max        ┆ 42672.0     │
└────────────┴─────────────┘


In [ ]:
df[var_name]

DistanceBus
i64
7528
2771
10663
4550
4754
…
5750
0
17015


In [ ]:
sns.histplot(df[var_name]);

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(x=df.iloc[:10000][var_name]);

In [ ]:
var_name = "estimateArrive"

In [ ]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df[var_name].describe())

### Paradas por línea

In [ ]:
# stops by line
stop_gr = df.groupby("line")["stop"].nunique().reset_index()
top_10_stop = stop_gr.sort_values(by="stop", ascending=False).head(10)

sns.barplot(data=top_10_stop, x="stop", y="line", orient="h", order=top_10_stop["line"])
plt.xlabel("Stops")
plt.ylabel("Line")
plt.title("Stops by Line")
plt.show()

### ETA de cada línea

In [ ]:
# estimateArrive and line

line_estimate_arrive_sorted = df.sort_values(by="estimateArrive", ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(
    x="estimateArrive",
    y="line",
    data=line_estimate_arrive_sorted,
    orient="h",
    order=line_estimate_arrive_sorted["line"].value_counts().head(10).index,
)
plt.xlabel("ETA (sec)")
plt.ylabel("Line")
plt.title("ETA by line")
plt.show()

### ETA por destino

In [ ]:
# ETA trends by destination
destination = df.groupby("destination")["estimateArrive"].mean().reset_index()
top_10_dest = destination.sort_values(by="estimateArrive", ascending=False).head(10)

sns.barplot(data=top_10_dest, x="estimateArrive", y="destination", orient="h")
plt.xlabel("AVG ETA (sec)")
plt.ylabel("Destination")
plt.title("ETA by Destination")
plt.show()

# MLIA

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import polars as pl

sns.set_palette("deep")
import warnings

warnings.filterwarnings("ignore")

In [2]:
ROOT_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))
DATA_PATH = os.path.join(ROOT_PATH, "data", "processed")
EMT_DATA_PATH = os.path.join(DATA_PATH, "emt")


In [3]:
lf = pl.scan_csv(os.path.join(EMT_DATA_PATH, "2024", "03", "emt_202403.csv"))

In [4]:
lf.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive
str,str,i64,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,bool,str,str,i64
"""2024-03-02""","""2024-03-02 07:…",579,"""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",1409
"""2024-03-02""","""2024-03-02 07:…",581,"""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",476
"""2024-03-02""","""2024-03-02 07:…",4855,"""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,false,null,null,2481
"""2024-03-02""","""2024-03-02 07:…",4857,"""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,false,null,null,967
"""2024-03-02""","""2024-03-02 07:…",5631,"""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,false,null,null,899


In [5]:
lf.explain(optimized=True)

'\n  Csv SCAN /home/mlia/proyectos/data-generation/data/processed/emt/2024/03/emt_202403.csv\n  PROJECT */19 COLUMNS'

# 1. Borramos la columna `positionTypeBus`

In [41]:
query_1  = lf.drop('positionTypeBus')

# 2. Creamos la PK

In [42]:
query = query_1.with_columns((pl.col('datetime').cast(pl.String)+"_B"+pl.col('bus').cast(pl.String)+"_L"+ pl.col('line').cast(pl.String)+"_S"+pl.col('stop').cast(pl.String)).alias('PK'))

In [43]:
query.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK
str,str,i64,str,i64,f64,f64,i64,str,i64,str,str,i64,i64,bool,str,str,i64,str
"""2024-03-02""","""2024-03-02 07:…",579,"""27""",33,-3.702641,40.40528,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",1409,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",581,"""27""",33,-3.689498,40.437068,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",476,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",4855,"""5""",33,-3.695955,40.435,10663,"""CHAMARTIN""",0,null,null,null,null,false,null,null,2481,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",4857,"""5""",33,-3.688575,40.430257,4550,"""CHAMARTIN""",0,null,null,null,null,false,null,null,967,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",5631,"""147""",33,-3.705854,40.428887,4754,"""BARRIO DEL PIL…",0,null,null,null,null,false,null,null,899,"""2024-03-02 07:…"


# 3. Filtramos para el 2 de marzo

In [44]:
query_2 = query.filter(pl.col('date') == "2024-03-02")

In [45]:
march_02 = query_2.head().collect()

In [46]:
march_02

date,datetime,bus,line,stop,positionBusLon,positionBusLat,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK
str,str,i64,str,i64,f64,f64,i64,str,i64,str,str,i64,i64,bool,str,str,i64,str
"""2024-03-02""","""2024-03-02 07:…",579,"""27""",33,-3.702641,40.40528,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",1409,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",581,"""27""",33,-3.689498,40.437068,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",476,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",4855,"""5""",33,-3.695955,40.435,10663,"""CHAMARTIN""",0,null,null,null,null,false,null,null,2481,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",4857,"""5""",33,-3.688575,40.430257,4550,"""CHAMARTIN""",0,null,null,null,null,false,null,null,967,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",5631,"""147""",33,-3.705854,40.428887,4754,"""BARRIO DEL PIL…",0,null,null,null,null,false,null,null,899,"""2024-03-02 07:…"


In [10]:
query_3 = query_2.group_by(by = [pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('datetime')]).count().filter(pl.col('count')==1)

In [52]:
query_4 = query_2.select('PK').group_by(by = [pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('datetime')]).count().filter(pl.col('count')>1)

In [53]:
query_4.collect()

ColumnNotFoundError: bus

Error originated just after this operation:
 SELECT [col("PK")] FROM
  FILTER [(col("date")) == (String(2024-03-02))] FROM

   WITH_COLUMNS:
   [[([([([([([(col("datetime").strict_cast(String)) + (String(_B))]) + (col("bus").strict_cast(String))]) + (String(_L))]) + (col("line").strict_cast(String))]) + (String(_S))]) + (col("stop").strict_cast(String))].alias("PK")]
     SELECT [col("date"), col("datetime"), col("bus"), col("line"), col("stop"), col("positionBusLon"), col("positionBusLat"), col("DistanceBus"), col("destination"), col("deviation"), col("StartTime"), col("StopTime"), col("MinimunFrequency"), col("MaximumFrequency"), col("isHead"), col("dayType"), col("strike"), col("estimateArrive")] FROM

        Csv SCAN /home/mlia/proyectos/data-generation/data/processed/emt/2024/03/emt_202403.csv
        PROJECT */19 COLUMNS

LogicalPlan had already failed with the above error; after failure, 1 additional operation was attempted on the LazyFrame

In [12]:
query_3.head().collect()

bus,line,stop,datetime,count
i64,str,i64,str,u32
2506,"""172""",4493,"""2024-03-02 12:…",1
2070,"""174""",5887,"""2024-03-02 09:…",1
4844,"""64""",3733,"""2024-03-02 11:…",1
2071,"""174""",51022,"""2024-03-02 13:…",1
2134,"""107""",1839,"""2024-03-02 11:…",1


In [12]:
query_day = lf.group_by(['datetime','bus','line','stop']).count().filter(pl.col('count')==1)

# Estimation of the passing time from ETAs
## 4.2.1 Filtering non real-time estimations

In [34]:
pk = march_02.group_by(['datetime','bus','line','stop']).count()
march_02 = pk.filter(pk['count']==1)
march_02.describe()


statistic,datetime,bus,line,stop,count
str,str,f64,str,f64,f64
"""count""","""560973""",560973.0,"""560973""",560973.0,560973.0
"""null_count""","""0""",0.0,"""0""",0.0,0.0
"""mean""",null,3468.028269,null,2445.628836,1.0
"""std""",null,2182.301776,null,4254.104916,0.0
"""min""","""2024-03-02 07:…",51.0,"""1""",29.0,1.0
"""25%""",null,2144.0,null,218.0,1.0
"""50%""",null,2505.0,null,1544.0,1.0
"""75%""",null,5538.0,null,3796.0,1.0
"""max""","""2024-03-02 15:…",9125.0,"""E1""",51023.0,1.0
